In [36]:
import gym
import tensorflow as tf
from tensorflow.python.framework import ops
ops.reset_default_graph()
import numpy as np
import gym  #requires OpenAI gym installed
import matplotlib.pyplot as plt
import numpy as np


envs = {
    'cartpole': gym.make('CartPole-v1'),
    'bipedal-walker-v2': gym.make('BipedalWalker-v2')

}

In [37]:
env = envs['bipedal-walker-v2']

input_dims = 24
state_placeholder = tf.placeholder(tf.float32, [None, input_dims]) 

def value_function(state):
    n_hidden1 = 400  
    n_hidden2 = 400
    n_outputs = 1
    
    with tf.variable_scope("value_network"):
        init_xavier = tf.contrib.layers.xavier_initializer()
        
        hidden1 = tf.layers.dense(state, n_hidden1, tf.nn.elu, init_xavier)
        hidden2 = tf.layers.dense(hidden1, n_hidden2, tf.nn.elu, init_xavier) 
        V = tf.layers.dense(hidden2, n_outputs, None, init_xavier)
    return V


def policy_network(state, n_outputs = 4):
    n_hidden1 = 40
    n_hidden2 = 40
    
    with tf.variable_scope("policy_network"):
        init_xavier = tf.contrib.layers.xavier_initializer()
        
        hidden1 = tf.layers.dense(state, n_hidden1, tf.nn.elu, init_xavier)
        hidden2 = tf.layers.dense(hidden1, n_hidden2, tf.nn.elu, init_xavier)
        mu = tf.layers.dense(hidden2, n_outputs, None, init_xavier)
        sigma = tf.layers.dense(hidden2, n_outputs, None, init_xavier)
        sigma = tf.nn.softplus(sigma) + 1e-5
        norm_dist = tf.contrib.distributions.Normal(mu, sigma)
        action_tf_var = tf.squeeze(norm_dist.sample(1), axis=0)
        action_tf_var = tf.clip_by_value(
            action_tf_var, env.action_space.low[0], 
            env.action_space.high[0])
    return action_tf_var, norm_dist

# define required placeholders
action_placeholder = tf.placeholder(tf.float32)
delta_placeholder = tf.placeholder(tf.float32)
target_placeholder = tf.placeholder(tf.float32)

action_tf_var, norm_dist = policy_network(state_placeholder)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    new_saver = tf.train.import_meta_graph('./tmp/pre_trained_step_1999_0.000020_0.000100.ckpt.meta')
    print(new_saver)
    new_saver.restore(sess, tf.train.latest_checkpoint('./tmp'))
    env.render()
    state = env.reset()   # state.shape -> (2,)
    steps = 0
    done = False
    while (not done):
        #Sample action according to current policy
        #action.shape = (1,4)
        steps += 1
        action  = sess.run(action_tf_var, feed_dict={
                      state_placeholder: state.reshape(1, -1)})
        #Execute action and observe reward & next state from E
        next_state, reward, done, _ = env.step(
                                np.squeeze(action, axis=0))
        state = next_state
        print(steps)

    env.close()

INFO:tensorflow:Restoring parameters from ./tmp/pre_trained_step_99_0.000002_0.000010.ckpt
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86


In [15]:
env.close()